# Question
Create Databases

Create the following databases (in a dataframe or HDF5 format) for each of the product categories Electronics, Movies and TV, Clothing Shoes and Jewelry, Cellphones and Accessories, Tools and Home Improvement, Beauty, and Baby:
* df_reviews_categoryname: whose columns are timestamp, productid, reviewerid, rating, review_text, review_summary
* df_products: whose columns are productid, title, imUrl, brand
* df_products_also_bought: indexed productid, contains also_bought column
* df_products_also_viewed: indexed productid, contains also_viewed column
* df_products_bought_together: indexed productid, contains bought_together column
* df_products_sales_rank: indexed by productid contains sales_rank
* df_products_categories: indexed by productid contains categories column


# Aproach

This script reads the reviews data and meta data by category and writes out hdf5 databases for each category with above dataframes as keys.

### Read (input) data location
* All data being read in for this file has been downloaded from here: http://jmcauley.ucsd.edu/data/amazon/
* Two types of data are read in for the categories described above
 * **reviews data** (downloaded the '5-core' files from the files section)
 * **meta data** (taken from the pre-category files section)
* The data are downloaded locally to the following folders, respectively
 > home/data_in/reviews_5core
 
 > home/data_in/meta_data

### Output data location
* All data is being written out to the folder
> home/data_out/databases/

### Output data format
* Data is being written out into hdf5 databases for each category.
* For example, baby.h5 contains the following keys as dataframes: df_reviews_categoryname, df_products, df_products_also_bought, df_products_also_viewed, df_products_bought_together, df_products_sales_rank and df_products_categories

### Process
* Each data frame asked for in the question is processed for all categories in a loop.
* I have defined functions for most processes.

### Some important notes
* df_products_also_bought, df_products_also_viewed, df_products_bought_together 
 * These have two columns each, productid and the list of productids associated. 
 * however, the second column is not a list, but a string. This string when being used needs to be split on ' ' to create a list. This was done for faster writing so pytables wouldn't pickle the columns. 
* productid = asin ( as found in reviews and metadata files)

### Time taken to run script
approx. 1 hr 15 mins to run

---

### IMPORTS

In [3]:
import json
import gzip
import pandas as pd
import glob
import timeit
import feather
import os
import shutil
import numpy as np
import time
# from pprint import pprint

### SCRIPT VARIABLES

In [13]:
test = True

refresh_all = True
df_reviews_categoryname_status = True

### FUNCTIONS

SCRIPT FUNCTIONS

In [4]:
####################
# SCRIPT FUNCTIONS #
####################

def parse(path):
    """
    This functions takes in a json file path 
    and reads it line by line as an actual
    because the lines are not real json 
    strings
    """
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    """
    input: path of a json file
    output: pd dataframe from json file
    """
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    df_out = pd.DataFrame.from_dict(df, orient='index')
    return(df_out)

def refresh_directory(dir_path):
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path)
    

##############################################
# FUNCTIONS USED IN PROCESSING REVIEWS FILES #
##############################################

def get_category(in_filename):
    """
    This function extracts the category name from the reviews file name
    input: reviews file name
    output: category string lower cased 
    """
    return (in_filename.split('/reviews_5core/reviews_')[1].split('_5.json.gz')[0].lower())


def get_df_reviews_categoryname(df):
    """
    This function creates the df_reviews_categoryname dataframe
    input: reviews data for a category, as a pd dataframe as read from json file
    output: dataframe with columns timestamp, productid, reviewerid, rating, review_text and review_summary
    """
    df_select = df[['unixReviewTime','asin','reviewerID','overall','reviewText','summary']]
    df_select.columns = ['timestamp', 'productid', 'reviewerid', 'rating', 'review_text', 'review_summary']
    df_null_asins_removed = df_select.dropna(axis=0, subset=['productid'])
    df_nulls_asins_removed_deduped = df_null_asins_removed.drop_duplicates(keep='first').reset_index(drop=True)
    return(df_null_asins_removed)


###############################################
# FUNCTIONS USED IN PROCESSING METADATA FILES #
###############################################

def get_category_from_metadata(in_filename):
    """
    This function extracts the category name from the metadata file name
    input: metadata file name
    output: category string lower cased 
    """
    return (in_filename.split('/meta_data/meta_')[1].split('.json.gz')[0].lower())

def get_df_from_metadata(df, columns_to_select, new_column_names):
    """
    This function simply selects target columns from a meta data pd dataframe and renames them
    input: meta data for a category, as a pd dataframe as read from json file
    output: dataframe with columns renamed by new_column_names
    """
    df_select = df[columns_to_select]
    df_select.columns = new_column_names
    df_null_asins_removed = df_select.dropna(axis=0, subset=['productid'])
    df_all_nulls_removed = df_select.dropna(axis=0,how='all').reset_index(drop=True)
    return(df_all_nulls_removed)

def get_dict_embeded_items(df,target_var, target_key):
    """
    This function takes a metadata pd dataframe with a column which contains dictionaries as values
    and filters out rows with desired keys only. It then converts the output
    to a dataframe 
    
    inputs: target dataframe, target var containing dict, target key to be pulled out
    outputs: a dataframe with columns product id, target key. 
    row output: 
        productid: productid value.
        target_key: a string of associated productids separated by spaces
    """
    asin_list= []
    target_list = []
    # loop through each row of the data frame
    for i in range(df.shape[0]):
        asin_value = df.loc[i,'asin'] # extract asin value from row
        target_dict = df.loc[i,target_var] # extract the dictionary from row
        if target_dict is np.nan:
            continue
        if target_key in target_dict:
            target_key_value = target_dict[target_key] # if the target key is in the dictionary, save
        else:
            target_key_value = np.nan
        asin_list.append(asin_value) # append values to list
        target_list.append(target_key_value) # append values to list
    df = pd.DataFrame({'productid':asin_list, target_key:target_list}) # convert to dataframe
    df.loc[:,target_key] = df[target_key].apply(lambda L: " ".join(str(x) for x in L) if L is not np.nan else L)
    df_nona= df.dropna(axis=0,how='any').reset_index(drop=True) # drop na rows
    return(df_nona)


def get_df_products_salesrank(df):
    """
    This function creates the df_products_salesrank dataframe. 
    input: metadata pd dataframe
    output: productid, categories = category of rank, salesRank = rank in category for productid
    
    process: the salesRank is a dictionary of category and rank. 
    The function explodes that dictionary into two columns of key and value
    """
    asin_list= []
    key_list = []
    value_list = []
    for i in range(df.shape[0]): # loop through each row of the dataframe
        asin_value = df.loc[i,'asin']
        target_dict = df.loc[i,'salesRank']
        if type(target_dict)==float: 
            continue # if it is a nan value, move to next loop
        for key in target_dict.keys():
            asin_list.append(asin_value) # add asin value to list
            key_list.append(key) # add key of the dictionary (category) to list
            value_list.append(target_dict[key]) # add the rank value to the list
    df = pd.DataFrame({'productid':asin_list, 'categories':key_list,'salesRank':value_list})
    df_nona= df.dropna(axis=0,how='any').reset_index(drop=True) # dropna
    return(df_nona)


def get_df_products_categories(df):
    """
    This function creates the df_products_categories dataframe
    input: metadata pd dataframe
    output: productid = asin values, categories = list of all categeries the asin belongs to
    process: the categories were lists of lists, this function creates a unique list for each asin
    """
    df_select = df[['asin','categories']]
    df_select.loc[:,'categories1'] = df_select.categories.apply(lambda l: list(set([item for sublist in l for item in sublist])))
    df_final =  df_select.drop(axis=1,columns=['categories'])
    df_final.columns = ['productid','categories']
    df_nona = df_final.dropna(axis=0,how='any').reset_index(drop=True)
    return(df_nona)

In [5]:
# REMOVE ALL VARIABLES
if refresh_all:
    refresh_directory('./data_out/databases/')

reviews_files_to_read = glob.glob('./data_in/reviews_5core/*.json.gz')
metadata_files_to_read = glob.glob('./data_in/meta_data/*.json.gz')

### 1) df_reviews_categoryname by category

In [16]:
for filename in reviews_files_to_read:
    # start timer
    start = timeit.default_timer() 
    
    # get data
    categoryname = get_category(filename)
    df_transformed = get_df_reviews_categoryname(df=getDF(filename))
    
    # write data
    output_filepath = './data_out/databases/' + categoryname + '.h5'
    df_transformed.to_hdf(output_filepath, key= 'df_reviews_categoryname', mode='a', format = 'fixed', data_columns = ['productid'])
    
    # end timer
    stop = timeit.default_timer() 
    
    # print results
    print(filename)
    print(df_transformed.shape)
    print('Time: ', stop - start)  
    print('\n')

./data_in/reviews_5core/reviews_Cell_Phones_and_Accessories_5.json.gz
(194439, 6)
Time:  12.418528391004656


./data_in/reviews_5core/reviews_Clothing_Shoes_and_Jewelry_5.json.gz
(278677, 6)
Time:  13.190927680996538


./data_in/reviews_5core/reviews_Electronics_5.json.gz
(1689188, 6)
Time:  96.25478842499433


./data_in/reviews_5core/reviews_Baby_5.json.gz
(160792, 6)
Time:  9.142972869994992


./data_in/reviews_5core/reviews_Tools_and_Home_Improvement_5.json.gz
(134476, 6)
Time:  7.274447283998597


./data_in/reviews_5core/reviews_Movies_and_TV_5.json.gz
(1697533, 6)
Time:  109.55919255800109




### 2) df_products

In [17]:
columns_to_select_ = ['asin','title','imUrl','brand']
new_column_names_ = ['productid','title','imUrl','brand']
key_ = 'df_products'

for filename in metadata_files_to_read:
    # start timer
    start = timeit.default_timer()

    # get data
    categoryname = get_category_from_metadata(filename)
    df_transformed = get_df_from_metadata(df=getDF(filename), columns_to_select = columns_to_select_, new_column_names = new_column_names_)
    
    # write data
    output_filepath = './data_out/databases/' + categoryname + '.h5'
    df_transformed.to_hdf(output_filepath, key= key_, mode='a', format = 'fixed', data_columns = ['productid'])
    
    # stop timer
    stop = timeit.default_timer()
    
    # print results
    print(filename)
    print(df_transformed.shape)
    print('Time: ', stop - start)  
    print('\n')

/Users/kanwal/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['productid', 'title', 'imUrl', 'brand']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


./data_in/meta_data/meta_Movies_and_TV.json.gz
(208321, 4)
Time:  23.62301140600175


./data_in/meta_data/meta_Electronics.json.gz
(498196, 4)
Time:  48.20840496199526


./data_in/meta_data/meta_Beauty.json.gz
(259204, 4)
Time:  32.33594568300032


./data_in/meta_data/meta_Tools_and_Home_Improvement.json.gz
(269120, 4)
Time:  27.00891852100176


./data_in/meta_data/meta_Baby.json.gz
(71317, 4)
Time:  9.63094223900407


./data_in/meta_data/meta_Cell_Phones_and_Accessories.json.gz
(346793, 4)
Time:  30.400116568998783


./data_in/meta_data/meta_Clothing_Shoes_and_Jewelry.json.gz
(1503384, 4)
Time:  182.36835953399714




### 3) df_products_also_bought

In [20]:
key_ = 'df_products_also_bought'
for filename in metadata_files_to_read:
    
    # start timer
    start = timeit.default_timer()

    # get data
    categoryname = get_category_from_metadata(filename)
    df = getDF(filename)
    df_transformed = get_dict_embeded_items(df=df,target_var='related', target_key='also_bought')
    
    # write data
    output_filepath = './data_out/databases/' + categoryname + '.h5'
    df_transformed.to_hdf(output_filepath, key= key_, mode='a', format = 'fixed', data_columns = ['productid'])
    
    # stop timer
    stop = timeit.default_timer()

    # print results
    print(filename)
    print(df_transformed.shape)
    print('Time: ', stop - start)  
    print('\n')

./data_in/meta_data/meta_Movies_and_TV.json.gz
(119220, 2)
Time:  27.285966067000118


./data_in/meta_data/meta_Electronics.json.gz
(141780, 2)
Time:  56.72587409699918


./data_in/meta_data/meta_Beauty.json.gz
(126419, 2)
Time:  38.10331394199602


./data_in/meta_data/meta_Tools_and_Home_Improvement.json.gz
(85937, 2)
Time:  31.628067198005738


./data_in/meta_data/meta_Baby.json.gz
(33847, 2)
Time:  12.582625779999944


./data_in/meta_data/meta_Cell_Phones_and_Accessories.json.gz
(123385, 2)
Time:  37.37732017199596


./data_in/meta_data/meta_Clothing_Shoes_and_Jewelry.json.gz
(661883, 2)
Time:  243.20014063400595




### 4) df_products_also_viewed

In [21]:
key_ = 'df_products_also_viewed'
for filename in metadata_files_to_read:
    
    # start timer
    start = timeit.default_timer()

    # get data
    categoryname = get_category_from_metadata(filename)
    df = getDF(filename)
    df_transformed = get_dict_embeded_items(df=df,target_var='related', target_key='also_viewed')
    
    # write data
    output_filepath = './data_out/databases/' + categoryname + '.h5'
    df_transformed.to_hdf(output_filepath, key= key_, mode='a', format = 'fixed', data_columns = ['productid'])
    
    # stop timer
    stop = timeit.default_timer()

    # print results
    print(filename)
    print(df_transformed.shape)
    print('Time: ', stop - start)  
    print('\n')

./data_in/meta_data/meta_Movies_and_TV.json.gz
(38798, 2)
Time:  30.600529074996302


./data_in/meta_data/meta_Electronics.json.gz
(190555, 2)
Time:  54.74302975700266


./data_in/meta_data/meta_Beauty.json.gz
(194249, 2)
Time:  38.83115368500148


./data_in/meta_data/meta_Tools_and_Home_Improvement.json.gz
(138443, 2)
Time:  32.687248663998616


./data_in/meta_data/meta_Baby.json.gz
(51741, 2)
Time:  11.749239247001242


./data_in/meta_data/meta_Cell_Phones_and_Accessories.json.gz
(87292, 2)
Time:  37.02678551799909


./data_in/meta_data/meta_Clothing_Shoes_and_Jewelry.json.gz
(960314, 2)
Time:  251.81925783900078




### 5) df_products_bought_together

In [22]:
key_ = 'df_products_bought_together'
for filename in metadata_files_to_read:
    
    # start timer
    start = timeit.default_timer()

    # get data
    categoryname = get_category_from_metadata(filename)
    df = getDF(filename)
    df_transformed = get_dict_embeded_items(df=df,target_var='related', target_key='bought_together')
    
    # write data
    output_filepath = './data_out/databases/' + categoryname + '.h5'
    df_transformed.to_hdf(output_filepath, key= key_, mode='a', format = 'fixed', data_columns = ['productid'])
    
    # stop timer
    stop = timeit.default_timer()

    # print results
    print(filename)
    print(df_transformed.shape)
    print('Time: ', stop - start)  
    print('\n')

./data_in/meta_data/meta_Movies_and_TV.json.gz
(81539, 2)
Time:  30.269841127999825


./data_in/meta_data/meta_Electronics.json.gz
(140898, 2)
Time:  55.679200569000386


./data_in/meta_data/meta_Beauty.json.gz
(112964, 2)
Time:  35.91843018500367


./data_in/meta_data/meta_Tools_and_Home_Improvement.json.gz
(102283, 2)
Time:  32.43215459200292


./data_in/meta_data/meta_Baby.json.gz
(31305, 2)
Time:  12.396068101996207


./data_in/meta_data/meta_Cell_Phones_and_Accessories.json.gz
(101462, 2)
Time:  37.524519144004444


./data_in/meta_data/meta_Clothing_Shoes_and_Jewelry.json.gz
(572794, 2)
Time:  234.1239114429991




### 6) df_products_sales_rank

In [23]:
key_ = 'df_products_sales_rank'
for filename in metadata_files_to_read:
    
    # start timer
    start = timeit.default_timer()

    # get data
    categoryname = get_category_from_metadata(filename)
    df = getDF(filename)
    df_transformed = get_df_products_salesrank(df)
    
    # write data
    output_filepath = './data_out/databases/' + categoryname + '.h5'
    df_transformed.to_hdf(output_filepath, key= key_, mode='a', format = 'fixed', data_columns = ['productid'])
    stop = timeit.default_timer()

    # print results
    print(filename)
    print(df_transformed.shape)
    print('Time: ', stop - start)  
    print('\n')

./data_in/meta_data/meta_Movies_and_TV.json.gz
(204907, 3)
Time:  34.98880069000006


./data_in/meta_data/meta_Electronics.json.gz
(107648, 3)
Time:  54.38227691600332


./data_in/meta_data/meta_Beauty.json.gz
(253985, 3)
Time:  36.520024053003


./data_in/meta_data/meta_Tools_and_Home_Improvement.json.gz
(51334, 3)
Time:  32.23155760100053


./data_in/meta_data/meta_Baby.json.gz
(36, 3)
Time:  11.269762977004575


./data_in/meta_data/meta_Cell_Phones_and_Accessories.json.gz
(339012, 3)
Time:  37.180118112002674


./data_in/meta_data/meta_Clothing_Shoes_and_Jewelry.json.gz
(1421649, 3)
Time:  283.67257153399987




### 7) df_products_categories

In [57]:
key_ = 'df_products_categories'
for filename in metadata_files_to_read:
    
    # start timer
    start = timeit.default_timer()

    # get data
    categoryname = get_category_from_metadata(filename)
    df = getDF(filename)
    df_transformed = get_df_products_categories(df)
    
    # write data
    output_filepath = './data_out/databases/' + categoryname + '.h5'
    df_transformed.to_hdf(output_filepath, key= key_, mode='a', format = 'fixed', data_columns = ['productid'])
    
    # stop timer
    stop = timeit.default_timer()

    # print results
    print(filename)
    print(df_transformed.shape)
    print('Time: ', stop - start)  
    print('\n')

/Users/kanwal/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/kanwal/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/kanwal/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-ty

./data_in/meta_data/meta_Movies_and_TV.json.gz
(208321, 2)
Time:  36.145316804002505


./data_in/meta_data/meta_Electronics.json.gz
(498196, 2)
Time:  72.35126653499901


./data_in/meta_data/meta_Beauty.json.gz
(259204, 2)
Time:  44.8357321509975


./data_in/meta_data/meta_Tools_and_Home_Improvement.json.gz
(269120, 2)
Time:  37.002529479999794


./data_in/meta_data/meta_Baby.json.gz
(71317, 2)
Time:  18.34138311400602


./data_in/meta_data/meta_Cell_Phones_and_Accessories.json.gz
(346793, 2)
Time:  40.507202704997326


./data_in/meta_data/meta_Clothing_Shoes_and_Jewelry.json.gz
(1503384, 2)
Time:  217.79500212400308




# end